# Calcul des diagrammes de plongement
Le but est de calculer le diagramme de plongement et de mettre en évidence une masse maximun. On se place dans le cas où l'étoile est homogène. Le système TOV s'intégre alors facilement et on a la forme de l'espace temps. 
Dans ce cas, la géométrie de l'espace temps ne dépend que de deux paramètres (R,M) ou (M, $\Xi$)

In [73]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets

%matplotlib notebook

### Intégration du sytème TOV dans le cas d'une sphère homogène

$$ m(r)	=\begin{cases}
M(r/R)^{3} & pour\,\,r\leq R\\
M & pour\,\,r\geq R
\end{cases}\\
P(r)	=\begin{cases}
\rho_{c}\frac{\sqrt{1-2\Xi(r/R){{}^2}}-\sqrt{1-2\Xi}}{3\sqrt{1-2\Xi}-\sqrt{1-2\Xi(r/R){{}^2}}} & pour\,\,r\leq R\\
0 & pour\,\,r\geq R
\end{cases}\\
\Phi(r)	=\begin{cases}
c{{}^2}ln\left(\frac{3\sqrt{1-2\Xi}-\sqrt{1-2\Xi(r/R){{}^2}}}{2}\right) & pour\,\,r\leq R\\
\frac{c{{}^2}}{2}ln\left(1-2\Xi\frac{R}{r}\right) & pour\,\,r\geq R
\end{cases} $$ 


La métrique en coordonnée de Schwarschild s'exprime avec : 
$$ N(r)=e^{\frac{\Phi(r)}{c^{2}}}$$ et $$ A(r)=\frac{1}{\sqrt{1-\frac{2Gm(r)}{rc^{2}}}}$$

In [69]:
c = 1
G = 1

def m(r, R, M):
    if r >=R : return M
    else : return M * (r/R)**3
    
def phi(r, R, M):
    xi = G*M/(R*c**2)
    if r >= R : return c**2 / 2 * np.log(1 - 2 * xi *R/r)
    else : return c**2 * np.log((3*np.sqrt(1 - 2*xi)-np.sqrt(1-2*xi * (r/R)**2))/2)

def P(r, R, M):
    xi = G*M/(R*c**2)
    if r <= R:
        rho = M/(4/3 * np.pi * R**3)
        return rho * c**2 * (np.sqrt(1-2*xi*(r/R)**2)-np.sqrt(1-2*xi))/(3*np.sqrt(1-2*xi) - np.sqrt(1-2*xi*(r/R)**2))
    else : return 0
    
def A(r, R, M):   
    if r >= R:
        R_c = 2 * G * M/c**2
        return -1/np.sqrt(1-R_c/r)
    elif r>0:
        m_r = m(r,R,M)
        #print(1-R_c/r, '   ', r)
        return -1/np.sqrt(1-2*G*m_r/(r * c**2))

### Graph dans le plan des courbes
On observe bien une compacité maximale au-delà de laquelle l'équilibre n'est plus possible pour $\Xi = \frac{4}{9}$

In [92]:

def graph(R, M):   
    n_radii = 100
    n_angles = 36

    xi = G*M/(R*c**2)
    print(xi, ' xi')
    # Make radii and angles spaces (radius r=0 omitted to eliminate duplication).
    radii = np.linspace(0, 10.0, n_radii)
    angles = np.linspace(0, 2*np.pi, n_angles, endpoint=False)

    # Repeat all angles for each radius.
    angles = np.repeat(angles[..., np.newaxis], n_radii, axis=1)

    # Convert polar (radii, angles) coords to cartesian (x, y) coords.
    # (0, 0) is manually added at this stage,  so there will be no duplicate
    # points in the (x, y) plane.
    x = np.append(1, (radii*np.cos(angles)).flatten())
    y = np.append(1, (radii*np.sin(angles)).flatten())

    # Compute z to make the pringle surface.
    z = np.vectorize(A)(np.sqrt(x**2 + y**2), R, M)

    fig = plt.figure()
    ax = fig.gca(projection='3d')

    ax.plot_trisurf(x, y, z, linewidth=0.2, antialiased=True)

    plt.show()

widgets.interact_manual(graph, R = widgets.fixed(1) , M = widgets.FloatLogSlider(base = 10, min=-4, max=0, step = 0.2,  description='xi'))   

interactive(children=(FloatLogSlider(value=1.0, description='xi', max=0.0, min=-4.0, step=0.2), Button(descrip…

<function __main__.graph(R, M)>